In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# API_ENDPOINT = os.environ["PAYGO_GPT_35_T_JP_ENDPOINT"]
# DEPLOYMENT = os.environ["PAYGO_GPT_35_T_DEPLOYMENT"]
# API_KEY = os.environ["PAYGO_GPT_35_T_JP_KEY"]

API_ENDPOINT = os.environ["PAYGO_GPT_4_AU_ENDPOINT"]
DEPLOYMENT = os.environ["PAYGO_GPT_4_AU_DEPLOYMENT"]
API_KEY = os.environ["PAYGO_GPT_4_AU_KEY"]

# API_ENDPOINT = os.environ["PTU_4o_SWC_ENDPOINT"]
# DEPLOYMENT = os.environ["PTU_4o_SWC_DEPLOYMENT"]
# API_KEY = os.environ["PTU_4o_SWC_KEY"]

## 4o PayGo Global
# API_ENDPOINT = os.environ["PAYGO_4o_SWC_ENDPOINT"]
# DEPLOYMENT = os.environ["PAYGO_4o_SWC_DEPLOYMENT"]
# API_KEY = os.environ["PAYGO_4o_SWC_KEY"]

## 4o PayGo Standard
# API_ENDPOINT = os.environ["PAYGO_4o_ENDPOINT"]
# DEPLOYMENT = os.environ["PAYGO_4o_DEPLOYMENT"]
# API_KEY = os.environ["PAYGO_4o_KEY"]

## 4o SWC GBB cluster
API_ENDPOINT = os.environ["GPT_4o_SWC_20240913_ENDPOINT"]
DEPLOYMENT = os.environ["GPT_4o_SWC_20240913_PTU_DEPLOYMENT"]
API_KEY = os.environ["GPT_4o_SWC_20240913_KEY"]

# Azure OpenAI API settings
API_ENDPOINT = f"{API_ENDPOINT}/openai/deployments/{DEPLOYMENT}/chat/completions?api-version=2024-06-01&max_tokens=500"
HEADERS = {
    "Content-Type": "application/json",
    "api-key": API_KEY
}

In [21]:
import requests

prompt = "Generate 500 tokens in Japanese about bank. Don't output anything else."

PAYLOAD = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 500,
    "temperature": 0.7,
    "top_p": 0.9,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "stream": False
}

def make_request():
    try:
        response = requests.post(API_ENDPOINT, headers=HEADERS, json=PAYLOAD)
        if response.status_code == 200:
            print(f"Success: {API_ENDPOINT} | {response.status_code} \n {response.headers} \n {response.json()}")
        else:
            print(f"Error: {API_ENDPOINT} | {response.status_code} \n {response.headers} \n {response.json()}")
    except Exception as e:
        print("Exception occurred:", e)

In [6]:
import requests
import time
import json
from datetime import datetime, timezone

prompt = "Generate 50 tokens in Japanese about bank. Don't output anything else."

PAYLOAD = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 500,
    "temperature": 0.7,
    "top_p": 0.9,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "stream": True
}

async def make_request_stream():
    request_time = time.time()
    request_time_str = datetime.now(timezone.utc).isoformat()
    try:
        response = requests.post(API_ENDPOINT, headers=HEADERS, json=PAYLOAD, stream=True)
        print(f"Request sent: {API_ENDPOINT} | {request_time_str}")
        if response.status_code == 200:
            ftt_time = None
            done_time = None
            
            for line in response.iter_lines():
                if line:
                    decoded_line = line.decode('utf-8').strip()
                    print(decoded_line)
                    if decoded_line.startswith("data: "):
                        data = decoded_line[6:]
                        if data == "[DONE]":
                            done_time = time.time()
                            done_time_str = datetime.now(timezone.utc).isoformat()
                            print(f"[DONE] received at: {done_time_str}")
                            print(f"Total time taken: {(done_time - request_time):.3f} seconds")
                            break
                        else:
                            try:
                                # First line is usually empty: data: {"choices":[{"content_filter_results":{},"delta":{"content":"","role":"assistant"},"finish_reason":null,"index":0,"logprobs":null}],"created":1726394751,"id":"chatcmpl-A7gQZ9vDUfpPD8ZtfkzpcrYQWxNmG","model":"gpt-4","object":"chat.completion.chunk","system_fingerprint":null}
                                # We only want to count first token time when there is really a token coming in the choices["delta"]["content"]

                                json_data = json.loads(data)
                                choices = json_data.get("choices", [])
                                if choices and "delta" in choices[0] and "content" in choices[0]["delta"]:
                                    content = choices[0]["delta"]["content"]
                                    if content:
                                        # print(f"Token received: {content}")
                                        if not ftt_time:
                                            ftt_time = time.time()
                                            ftt_time_str = datetime.now(timezone.utc).isoformat()
                                            print(f"First token received at: {ftt_time_str}")
                                            print(f"Time taken to first token: {(ftt_time - request_time):.3f} seconds")
                            except json.JSONDecodeError:
                                print(f"Failed to decode JSON: {data}")
                    if not ftt_time:
                        print(f"Line received before first token: {decoded_line}")
            if not ftt_time:
                print("No tokens received")
            if not done_time:
                if ftt_time:
                    print(f"First token received at: {ftt_time_str}")
                    print(f"Time taken to first token: {(ftt_time - request_time):.3f} seconds")
                print("No [DONE] received")
        else:
            print(f"Error: Status: {response.status_code} \n {response.headers} \n {response.json()}")
    except Exception as e:
        print("Exception occurred:", e)
                        

In [7]:
# stream
import asyncio
await make_request_stream()

Request sent: https://gbb-ea-openai-swedencentral-gpt4o-a.openai.azure.com/openai/deployments/gpt-4o-2024-05-13-ptu/chat/completions?api-version=2024-06-01&max_tokens=500 | 2024-09-16T13:04:15.649432+00:00
data: {"id":"chatcmpl-A85gmJLqonnBDX3HRGLYnX7saZOEV","object":"chat.completion.chunk","created":1726491856,"model":"gpt-4o-2024-05-13","system_fingerprint":"fp_80a1bad4c7","choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}
Line received before first token: data: {"id":"chatcmpl-A85gmJLqonnBDX3HRGLYnX7saZOEV","object":"chat.completion.chunk","created":1726491856,"model":"gpt-4o-2024-05-13","system_fingerprint":"fp_80a1bad4c7","choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}]}
data: {"id":"chatcmpl-A85gmJLqonnBDX3HRGLYnX7saZOEV","object":"chat.completion.chunk","created":1726491856,"model":"gpt-4o-2024-05-13","system_fingerprint":"fp_80a1bad4c7","choices":[{"index":0,"delta":{"cont

In [16]:
make_request() # 35-t doesn't have enough context window

Success: https://benchmark-20240808-australia-east.openai.azure.com/openai/deployments/gpt-4-0613/chat/completions?api-version=2024-06-01&max_tokens=500 | 200 
 {'Cache-Control': 'no-cache, must-revalidate', 'Content-Length': '2260', 'Content-Type': 'application/json', 'access-control-allow-origin': '*', 'apim-request-id': 'c7991e22-235b-4c72-ac93-6a1cad881345', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'Australia East', 'x-ratelimit-remaining-requests': '39', 'x-ratelimit-remaining-tokens': '39500', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'x-request-id': 'd02e094c-f246-4336-9b67-b3c012c2e8b4', 'x-ms-client-request-id': 'c7991e22-235b-4c72-ac93-6a1cad881345', 'azureml-model-session': 'd017-20240819155551', 'Date': 'Sun, 15 Sep 2024 06:29:32 GMT'} 
 {'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'saf

PTU

In [36]:
make_request()

Success: {'id': 'chatcmpl-9uE0tkHY9NzuZNPwv78Py7sjWZh9S', 'object': 'chat.completion', 'created': 1723187263, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '銀行業務において、お客様に最高のサービスを提供するためには、以下のような手続きやプロセスが必要です。\n\n### 1. 口座開設手続き\n- **申込書の記入**: お客様は銀行が提供する申込書に必要事項を記入します。\n- **本人確認書類の提出**: 身分証明書（運転免許証、パスポートなど）を提出し、銀行が本人確認を行います。\n- **初回入金**: 口座開設後、現金または振込で初回入金を行います。\n\n### 2. インターネットバンキングの登録\n- **ウェブサイトへのアクセス**: 銀行のウェブサイトにアクセスします。\n- **ユーザーIDとパスワードの設定**: 自分のユーザーIDとパスワードを設定します。\n- **認証コードの受信と入力**: 銀行から送られる認証コードを受け取り、ウェブサイトに入力します。\n\n### 3. ローン申'}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 2935, 'completion_tokens': 250, 'total_tokens': 3185}, 'system_fingerprint': 'fp_abc28019ad'}


PayGo Global

In [41]:
make_request()

Success: {'id': 'chatcmpl-9uE2Cc0Wk5RmYL0WU7bEvrLgqdYYE', 'object': 'chat.completion', 'created': 1723187344, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '銀行業務においてお客様に最高のサービスを提供するためには、様々な手続きやプロセスが必要です。以下に、一般的な銀行業務のフローについて詳しく説明します。\n\n### 口座開設の手続き\n1. **申込書の記入**: お客様は銀行が提供する申込書に必要事項を記入します。\n2. **本人確認書類の提出**: お客様は身分証明書（運転免許証、パスポートなど）を提出し、銀行が本人確認を行います。\n3. **初回入金**: 口座開設後、初回の入金が必要です。お客様は現金または振込で初回入金を行います。\n\n### インターネットバンキングの登録\n1. **ウェブサイトへのアクセス**: お客様は銀行のウェブサイトにアクセスします。\n2. **ユーザーIDとパスワードの設定**: お客様は自分のユーザーIDとパスワードを設定します。\n3. **認証コードの'}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 2935, 'completion_tokens': 250, 'total_tokens': 3185}, 'system_fingerprint': 'fp_abc28019ad'}


4o PayGo Standard

In [49]:
make_request()

Success: {'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'length', 'index': 0, 'logprobs': None, 'message': {'content': '銀行業務においてお客様に最高のサービスを提供するためには、以下のような様々な手続きやプロセスが必要です。\n\n### 口座開設\n1. **申込書の記入**: お客様が銀行が提供する申込書に必要事項を記入します。\n2. **本人確認書類の提出**: 身分証明書（運転免許証、パスポートなど）を提出し、銀行が本人確認を行います。\n3. **初回入金**: 口座開設後、現金または振込で初回入金を行います。\n\n### インターネットバンキングの登録\n1. **ウェブサイトへのアクセス**: 銀行のウェブサイトにアクセスします。\n2. **ユーザーIDとパスワードの設定**: 自分のユーザーIDとパスワードを設定します。\n3. **認証コードの受信と入力**: 銀行から送られる認証コードを受け取り、ウェブサイトに入力します。\n\n### 各種サービスの提供', 'role': 'assistant'}}], 'created': 1723206237, 'id': 'chatcmpl-9uIwvIuU5UWpjMQTvFH0Bs8KH3cc0', 'model': 'gpt-4o-2024-05-13', 'object': 'c

In [8]:
from datetime import datetime, timezone
datetime.now(timezone.utc).isoformat()

'2024-09-13T11:38:04.691832+00:00'

## OpenAI

In [55]:
API_ENDPOINT = os.getenv("OPENAI_API_ENDPOINT")
API_KEY = os.getenv("OPENAI_API_KEY")

In [57]:
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

In [58]:
import requests

prompt = "Generate 50 tokens in Japanese about bank. Don't output anything else."


PAYLOAD = {
    "model": "gpt-4o",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 50,
    "temperature": 0.7,
    "top_p": 0.9,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "stream": False
}

def make_request():
    try:
        response = requests.post(API_ENDPOINT, headers=HEADERS, json=PAYLOAD)
        if response.status_code == 200:
            print(f"Success: {API_ENDPOINT} | {response.status_code} \n {response.headers} \n {response.json()}")
        else:
            print(f"Error: {API_ENDPOINT} | {response.status_code} \n {response.headers} \n {response.json()}")
    except Exception as e:
        print("Exception occurred:", e)

In [59]:
make_request()

Success: https://api.openai.com/v1/chat/completions | 200 
 {'Date': 'Mon, 16 Sep 2024 06:38:36 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-expose-headers': 'X-Request-ID', 'openai-organization': 'user-cah75j5ctcmleczenoknaz2k', 'openai-processing-ms': '596', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15552000; includeSubDomains; preload', 'x-ratelimit-limit-requests': '500', 'x-ratelimit-limit-tokens': '30000', 'x-ratelimit-remaining-requests': '499', 'x-ratelimit-remaining-tokens': '29923', 'x-ratelimit-reset-requests': '120ms', 'x-ratelimit-reset-tokens': '154ms', 'x-request-id': 'req_68a10fb9378dd7447a826d79f2a58eb1', 'CF-Cache-Status': 'DYNAMIC', 'Set-Cookie': '__cf_bm=KzVoKGnOO3kSc2ywHAex0SiZcrJifMdb0jNQXZJBK1k-1726468716-1.0.1.1-rZbPzL5ODO4vX.wf6Vf4rpdjyeRT8D.enpAGFKfHJyqmpgGaCMHDQb.mIUYdD7_bl.uG7XUT7fGedmrLPCLIuw; path=/; expires=Mon, 16-Sep-24 07:08:36 GMT; domain=.api.openai.c